In [3]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
np.random.seed(42)
import pprint
import s3fs

import sys
sys.path.append("../../")
from robust_pca.benchmark import cross_validation, comparator
from robust_pca.imputations import models

In [5]:
#path = "s3://aifluence-data2/data/02_intermediate/chatelet_pdt/H.parq"
#df = pd.read_parquet(path, engine="pyarrow")
df = pd.read_parquet("../../../data/H.parq", engine="pyarrow")

**data preparation**

In [77]:
def nan_rule(df, columns, hours):
    """replace nan values by zero if "true" zero (e.g. no train during this period)
    Parameters
    ----------
    df : pd.DataFrame
        "spatiotemporal" dataframe
    columns : List[str]
        list of column names we have to replace nan by zero
    hours : List[str]
        list of hours for which we know there is no train -> true zeros
    """
    data = df.copy()
    data.loc[data.index.get_level_values("datetime").hour.isin(hours), columns] = data.loc[data.index.get_level_values("datetime").hour.isin(hours), columns].fillna(0)
    return data


data = df.reset_index()
#del data["datetime"]
#data = data.rename(columns={"datetime_theo_pdt":"datetime"})
data = data.sort_values(by=["station", "datetime", "position"])
data = data.set_index(["line","station", "datetime", "direction"])
data = nan_rule(data, ["load"], [0, 1, 2, 3])
data = data.reset_index()

In [94]:
models_to_test = [
  # models.ImputeByMean(["datetime.dt.hour", "direction"]), 
  # models.ImputeByMedian(["datetime.dt.hour", "datetime.dt.dayofweek", "direction"]), 
  # models.RandomImpute(),
  # models.ImputeLOCF(["direction"]),
  # models.ImputeNOCB(["direction"]),
  # models.ImputeKNN(),
  models.ImputeKNN(period=1, method="temporal"),
#   models.ImputeProphet(**{
#               "weekly_seasonality": True, 
#               "yearly_seasonality": True,
#               "interval_width": 0.95,
#               })
# models.ImputeRPCA(),
]

search_params = {
  "ImputeKNN": {"k": {"min":2, "max":3, "type":"Integer"}},
  "ImputeProphet": {
          "daily_seasonality": {"categories": [True, False], "type":"Categorical"},
        },
  "ImputeRPCA": {
    "lambda": {"min": 1e-3, "max": 1e3, "type":"Real"},
    "tau": {"min": 1e-3, "max": 1e3, "type":"Real"},
  }
}

line = "H"
station = "VALMONDOIS"
direction = "Paris"
dataset = data[(data["line"]==line) & (data["station"]==station) & (data["direction"]==direction)]
cols_to_impute = ["load"]

# dataset = dataset.set_index(["line", "station", "datetime", "direction"])
# dataset = dataframe with multiindex and at least, columns in cols_to_impute

dataset = dataset.set_index(["line","station", "datetime", "direction"])
#dataset = dataset.set_index(["datetime", "direction"])
# dataset = dataframe with multiindex and at least, columns in cols_to_impute

prop_nan = 0.05

In [95]:
dataset.head()

datetime_theo_pdt   train  \
line station    datetime            direction                               
H    VALMONDOIS 2019-01-01 04:46:52 Paris     2019-01-01 04:46:00  124600   
                2019-01-01 04:46:54 Paris     2019-01-01 04:46:00  124600   
                2019-01-01 06:18:32 Paris     2019-01-01 06:16:00  124606   
                2019-01-01 06:18:38 Paris     2019-01-01 06:16:00  124606   
                2019-01-01 06:47:52 Paris     2019-01-01 06:46:00  124612   

                                               position  date_count  \
line station    datetime            direction                         
H    VALMONDOIS 2019-01-01 04:46:52 Paris             2  2019-01-01   
                2019-01-01 04:46:54 Paris             1  2019-01-01   
                2019-01-01 06:18:32 Paris             2  2019-01-01   
                2019-01-01 06:18:38 Paris             1  2019-01-01   
                2019-01-01 06:47:52 Paris             1  2019-01-01   

                                               n_coaches mission  \
line station    datetime            direction                      
H    VALMONDOIS 2019-01-01 04:46:52 Paris              2    AVOP   
                2019-01-01 04:46:54 Paris              2    AVOP   
                2019-01-01 06:18:32 Paris              2    AVOP   
                2019-01-01 06:18:38 Paris              2    AVOP   
                2019-01-01 06:47:52 Paris              2    AVOP   

                                                datetime_theo_cht  loading  \
line station    datetime            direction                                
H    VALMONDOIS 2019-01-01 04:46:52 Paris     2019-01-01 04:46:00      1.0   
                2019-01-01 04:46:54 Paris     2019-01-01 04:46:00      0.0   
                2019-01-01 06:18:32 Paris     2019-01-01 06:16:00      2.0   
                2019-01-01 06:18:38 Paris     2019-01-01 06:16:00      1.0   
                2019-01-01 06:47:52 Paris     2019-01-01 06:46:00      1.0   

                                               unloading  load  \
line station    datetime            direction                    
H    VALMONDOIS 2019-01-01 04:46:52 Paris            0.0   8.0   
                2019-01-01 04:46:54 Paris            0.0  17.0   
                2019-01-01 06:18:32 Paris            0.0  43.0   
                2019-01-01 06:18:38 Paris            0.0  37.0   
                2019-01-01 06:47:52 Paris            0.0  27.0   

                                                    origin_cht  \
line station    datetime            direction                    
H    VALMONDOIS 2019-01-01 04:46:52 Paris      Persan-Beaumont   
                2019-01-01 04:46:54 Paris      Persan-Beaumont   
                2019-01-01 06:18:32 Paris      Persan-Beaumont   
                2019-01-01 06:18:38 Paris      Persan-Beaumont   
                2019-01-01 06:47:52 Paris      Persan-Beaumont   

                                                              destination_cht  \
line station    datetime            direction                                   
H    VALMONDOIS 2019-01-01 04:46:52 Paris      Paris Nord via Ermont Eaubonne   
                2019-01-01 04:46:54 Paris      Paris Nord via Ermont Eaubonne   
                2019-01-01 06:18:32 Paris      Paris Nord via Ermont Eaubonne   
                2019-01-01 06:18:38 Paris      Paris Nord via Ermont Eaubonne   
                2019-01-01 06:47:52 Paris      Paris Nord via Ermont Eaubonne   

                                                        ORIGIN  \
line station    datetime            direction                    
H    VALMONDOIS 2019-01-01 04:46:52 Paris      PERSAN BEAUMONT   
                2019-01-01 04:46:54 Paris      PERSAN BEAUMONT   
                2019-01-01 06:18:32 Paris      PERSAN BEAUMONT   
                2019-01-01 06:18:38 Paris      PERSAN BEAUMONT   
                2019-01-01 06:47:52 Paris      PERSAN BEAUMONT   

                             

**Comparator**

In [102]:
comparison = comparator.Comparator(
          dataset,
          prop_nan, 
          models_to_test, 
          cols_to_impute,
          search_params=search_params,
)
results = comparison.compare()

pp = pprint.PrettyPrinter(depth=4)
pp.pprint(results)

ImputeKNN
in fit_transform_col: 3
                                                   load
line station    datetime            direction          
H    VALMONDOIS 2019-01-01 04:46:52 Paris       8.00000
                2019-01-01 04:46:54 Paris      17.00000
                2019-01-01 06:18:32 Paris      43.00000
                2019-01-01 06:18:38 Paris      37.00000
                2019-01-01 06:47:52 Paris      27.00000
...                                                 ...
                NaT                 Paris      38.99398
                                    Paris      38.99398
                                    Paris      38.99398
                                    Paris      38.99398
                                    Paris      38.99398

[44963 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
                                                load
line station    datetime            direction       
H    VALMONDOIS 2019-01-01 04:46:52 Paris      False
               

KeyboardInterrupt: 